### 載入SparkR

In [ ]:
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sparkR.session(master = "local[*]")

### 讀入csv檔產生SparkDataFrame

In [ ]:
data <- file.path('', 'home','spark','pu_workshop','kmeans_data.csv')

In [ ]:
data_df <- read.df(
    data, 
    header='true', 
    source = "com.databricks.spark.csv", 
    inferSchema='true')

In [ ]:
head(data_df)

In [ ]:
printSchema(data_df)

### 使用spark.kmeans()進行training

In [ ]:
kmeansModel <- spark.kmeans(data_df, ~ x + y,k = 3)

In [ ]:
summary(kmeansModel)

### 使用ggplot2將資料點畫出

In [ ]:
library(ggplot2)

In [ ]:
p1 <- collect(select(filter(data_df, "cluster = 1"), "x", "y"))
p2 <- collect(select(filter(data_df, "cluster = 2"), "x", "y"))
p3 <- collect(select(filter(data_df, "cluster = 3"), "x", "y"))

In [ ]:
c1 <- ggplot(data=p1, aes(x=x, y=y)) + geom_point(aes(color="red"))
c2 <- c1 +  geom_point(data = p2, aes(x=x, y= y, color = "blue"))
c3 <- c2 + geom_point(data = p3, aes(x=x, y= y, color = "green"))
c3

In [ ]:
centers <- data.frame(summary(kmeansModel)$coefficients)
centers